# Testing statistics

In [ ]:
import numpy as np
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import statsmodels.api as sm
import statistics
import os 
from scipy.stats import norm

## The irreproducibility of p-values

P-values are themselves rather irreproducible. This is a bit hard to imagine, so we will have a look t that.  

First, lets simulate data. We want to have two normal distributions, which are supposed to be borderline significantly different with a significance level of alpha = 0.05  

In [ ]:
mean1 = 0
mean2 = 10
sd= 10 
n=10

s1 = np.random.normal(mean1, sd,n)
s2 = np.random.normal(mean2, sd,n)

plt.hist(s1,alpha=0.5)
plt.hist(s2,alpha=0.5)
plt.xlabel("value")
plt.ylabel("density")
plt.axvline(statistics.mean(s1), color="blue")
plt.axvline(statistics.mean(s2), color="red")

st.ttest_ind(s1,s2)

Now lets do this many times to have a look how often we get the p-value.

In [ ]:
pvals = []

for _ in range (1000):
    np.random.seed()
    samp1 = np.random.normal(mean1, sd, n)
    samp2 = np.random.normal(mean2, sd, n)
    new_test = st.ttest_ind(samp1,samp2) 
    pvals.append(new_test[1])

#Can you simplify the loop?     
    
#Lets plot this
plt.hist(-np.log10(pvals),bins=50)
plt.axvline(-np.log10(0.05), color="black")
plt.xlabel("-log10(p)")


We are indeed at the significance level most of the time, but at the same time we would frquently call our results "not significant" and sometimes "highly significant".  
Does this become better with increased sample size? 

In [ ]:
n=20

s1 = np.random.normal(mean1, sd,n)
s2 = np.random.normal(mean2, sd,n)

plt.hist(s1,alpha=0.5)
plt.hist(s2,alpha=0.5)
plt.xlabel("value")
plt.ylabel("density")
plt.axvline(statistics.mean(s1), color="blue")
plt.axvline(statistics.mean(s2), color="red")

In [ ]:
pvals = []

for _ in range (1000):
    np.random.seed()
    samp1 = np.random.normal(mean1, sd, n)
    samp2 = np.random.normal(mean2, sd, n)
    new_test = st.ttest_ind(samp1,samp2) 
    pvals.append(new_test[1])

    
    
#Lets plot this
plt.hist(-np.log10(pvals),bins=50)
plt.axvline(-np.log10(0.05), color="black")
plt.xlabel("-log10(p)")

Indeed it does. The "power" of our analysis increases, but the range is still very wide. And this does not really change! 

In [ ]:
n=100

s1 = np.random.normal(mean1, sd,n)
s2 = np.random.normal(mean2, sd,n)

plt.hist(s1,alpha=0.5)
plt.hist(s2,alpha=0.5)
plt.xlabel("value")
plt.ylabel("density")
plt.axvline(statistics.mean(s1), color="blue")
plt.axvline(statistics.mean(s2), color="red")



In [ ]:
pvals = []

for _ in range (1000):
    np.random.seed()
    samp1 = np.random.normal(mean1, sd, n)
    samp2 = np.random.normal(mean2, sd, n)
    new_test = st.ttest_ind(samp1,samp2) 
    pvals.append(new_test[1])

   
    
#Lets plot this
plt.hist(-np.log10(pvals),bins=50)
plt.axvline(-np.log10(0.05), color="black")
plt.xlabel("-log10(p)")

A good way to visualise this is by plotting the the p-value AND the effect size. In our case the difference between the mean 

In [ ]:
n=10

pvals = []
diffs = []

for _ in range (1000):
    np.random.seed()
    samp1 = np.random.normal(mean1, sd, n)
    samp2 = np.random.normal(mean2, sd, n)
    new_test = st.ttest_ind(samp1,samp2) 
    diff = statistics.mean(samp2)-statistics.mean(samp1)
    diffs.append(diff)
    pvals.append(new_test[1])

df = pd.DataFrame({'diff': diffs, 'pvals': pvals})

print(df)
    
#Lets plot this
plt.scatter(df["diff"], -np.log10(df["pvals"]),  alpha=0.5)
plt.xlabel("difference of the mean")
plt.ylabel("-log10(p)")
plt.axhline(-np.log10(0.05), color="grey")
plt.axvline(0, color="grey")



Such plots are called volcano plots.

## Leukocyte counts 

![blood.jpg](attachment:blood.jpg)  

Adapted from A. Rad and M. Häggström. CC-BY-SA 3.0 license

We are doing an experiment to see whether a COVID-19 infection changes the white blood cell counts in comparison to normal. 
From the literature (i.e. internet) we know that a "normal" whilte blood cell count is between 4000 and 11000 white blood cells per microliter. We measure this in 100 patients and 100 controls and note down the data in "white blood cell counts per microliter"  
Another disease where we expect a difference is chronic lymphocytic leukemia (CLL), so we decide to also collect these data from 100 patients.  

Lets import these data:

In [ ]:
dat = pd.read_csv('https://raw.githubusercontent.com/BiAPoL/Bio-image_Analysis_with_Python/main/data/leukocyte_counts.csv')

print(dat)

Lets first focus on COVID19 and the first step: Visualising the data and summary statistics

In [ ]:
plt.hist(dat['healthy'],bins = 50,alpha=0.5)
plt.hist(dat['COVID19'],bins = 50,alpha=0.5)
plt.xlabel("value")
plt.ylabel("density")
plt.axvline(statistics.mean(dat['healthy']), color="blue")
plt.axvline(statistics.mean(dat['COVID19']), color="red")

How are these data distributed?

In [ ]:
dat["healthy"].describe()

In [ ]:
dat["COVID19"].describe()

Can we perform a t-test on these data?

In [ ]:
st.ttest_ind(dat['healthy'],dat['COVID19'])

How do you interpret this result?  
Were you allowed to perform a t-test in the first place?   
If yes, why? If not, why not? What would be an alternative?


Is there a difference in Leukocyte counts in chronic lymphocytic leukemia? 

For the COVID cases you are still unsure, whether you had to reject H0, because you were just not looking at enough patients. Therefore you decide on a little simulation experiment: If the effect size and distributions were true, how many patients and controls would you estimate to look at for it to become significant with a significance level of alpha = 0.05?   